# Analyse topological divergences and the baselines for Lyapunov estimation

In [1]:
import pandas as pd
import os
import pickle

RESULTS_DIR = "outputs/data/divergence_results"
pkl_files = [
    f
    for f in os.listdir(RESULTS_DIR)
    if f.endswith(("_500.pkl", "_750.pkl", "_1000.pkl"))
]

data_frames = []

for pkl_file in pkl_files:
    # Load the dictionary from the .pkl file
    with open(f"{RESULTS_DIR}/{pkl_file}", "rb") as file:
        data_dict = pickle.load(file)

    # Convert dictionary into a Pandas DataFrame
    data_frame = pd.DataFrame(list(data_dict.items()), columns=["Keys", "Values"])

    # Split the tuple keys into separate columns
    data_frame[["Measure", "System", "Length"]] = pd.DataFrame(
        data_frame["Keys"].tolist(), index=data_frame.index
    )

    # Drop the original keys column
    data_frame = data_frame.drop(["Keys"], axis=1)

    # Use the new columns as a multi-index
    # data_frame.set_index(["measure", "system", "length"], inplace=True)

    # Add the dataframe to the list
    data_frames.append(data_frame)

# Concatenate the data frames
merged_data = pd.concat(data_frames)

# Split values dictionaries into separate columns and drop original column
merged_data = pd.concat(
    [merged_data.drop(["Values"], axis=1), merged_data["Values"].apply(pd.Series)],
    axis=1,
)

# Further split the two correlation columns into the coefficient and pvalues
merged_data[["Spearman Rho (All)", "Spearman p (All)"]] = merged_data[
    "spearmanr"
].apply(pd.Series)
merged_data[["Spearman Rho (Chaos)", "Spearman p (Chaos)"]] = merged_data[
    "pos_spearmanr"
].apply(pd.Series)

# Forget the original columns that were split
merged_data = merged_data.drop(columns=["spearmanr", "pos_spearmanr"])

# Forget useless divergence
ignore_measures = (
    "interleaving_length_0.5_None divergence",
    "interleaving_length_0.4_None divergence",
    "interleaving_length_0.3_None divergence",
    "interleaving_length_0.2_None divergence",
    "interleaving_length_0.1_None divergence",
    "cophenetic_length",
    "cophenetic_reverse_length",
)
merged_data = merged_data[~merged_data["Measure"].isin(ignore_measures)]

# Ensure the system names are consistent across records
merged_data["System"] = merged_data["System"].replace("Hénon", "Henon")
merged_data["System"] = merged_data["System"].replace("henon", "Henon")
merged_data["System"] = merged_data["System"].replace("ikeda", "Ikeda")
merged_data["System"] = merged_data["System"].replace("logistic", "Logistic")
merged_data["System"] = merged_data["System"].replace("tinkerbell", "Tinkerbell")

# Create readable column names
merged_data.rename(
    columns={
        "classification_f1": "Random Forest F1 Score",
        "regression_neg_mean_absolute": "Random Forest MSE (All)",
        "pos_regression_neg_mean_absolute": "Random Forest MSE (Chaos)",
        "regression_neg_mean_absolute_poly": "kNN MSE (All)",
        "pos_regression_neg_mean_absolute_poly": "kNN MSE (Chaos)",        
    },
    inplace=True
)


In [2]:
merged_data.head()

,Measure,System,Length,Random Forest F1 Score,Random Forest MSE (All),Random Forest MSE (Chaos),kNN MSE (All),kNN MSE (Chaos),Spearman Rho (All),Spearman p (All),Spearman Rho (Chaos),Spearman p (Chaos)
0,cophenetic divergence,Henon,500,0.770146,-0.088087,-0.065053,-0.085484,-0.059962,0.736482,1.396494e-86,0.746799,3.699274e-43
1,cophenetic_reverse divergence,Henon,500,0.784740,-0.118038,-0.093484,-0.093558,-0.067900,0.798787,5.628399e-112,0.831906,1.592406e-61
2,interleaving_0.5_None divergence,Henon,500,0.779450,-0.094765,-0.092251,-0.085155,-0.076143,0.795286,2.552484e-110,0.606278,5.620223e-25
4,interleaving_edge_0.5_None divergence,Henon,500,0.782874,-0.102633,-0.085643,-0.090005,-0.078395,0.790853,2.875452e-108,0.699864,6.632495e-36
5,interleaving_0.4_None divergence,Henon,500,0.790967,-0.119352,-0.084975,-0.086889,-0.076964,0.769395,5.357184e-99,0.585371,5.242113e-23


In [3]:
measure_long_name_map = {
    'cophenetic divergence': 'DMT Cophenetic Divergence',
    'cophenetic_reverse divergence': 'DMT Cophenetic (Reverse) Divergence',
    'interleaving_0.5_None divergence': 'PLMT Interleaving (0.5) Divergence',
    'interleaving_0.4_None divergence': 'PLMT Interleaving (0.4) Divergence',
    'interleaving_0.3_None divergence': 'PLMT Interleaving (0.3) Divergence',
    'interleaving_0.2_None divergence': 'PLMT Interleaving (0.2) Divergence',
    'interleaving_0.1_None divergence': 'PLMT Interleaving (0.1) Divergence',
    'interleaving_edge_0.5_None divergence': 'PLMT Interleaving (0.5) Divergence (Edge-Normalised)',
    'interleaving_edge_0.4_None divergence': 'PLMT Interleaving (0.4) Divergence (Edge-Normalised)',
    'interleaving_edge_0.3_None divergence': 'PLMT Interleaving (0.3) Divergence (Edge-Normalised)',
    'interleaving_edge_0.2_None divergence': 'PLMT Interleaving (0.2) Divergence (Edge-Normalised)',
    'interleaving_edge_0.1_None divergence': 'PLMT Interleaving (0.1) Divergence (Edge-Normalised)',
    'bottleneck divergence': 'Persistence Bottleneck Divergence',
    'wasserstein divergence': 'Persistence Wasserstein Divergence',
    '$k$-NN graph $M(D)$': 'kNN Graph PD Homology Class Ratio - M(D)',
    '$k$-NN graph $P(D)$': 'kNN Graph PD Maximum Persistence - P(D)',
    '$k$-NN graph $E\'(D)$': 'kNN Graph PD Normalised Persistent Entropy - E\'(D)',
    'Ordinal graph $M(D)$': 'Ordinal Partition Network Graph PD Homology Class Ratio - M(D)',
    'Ordinal graph $P(D)$': 'Ordinal Partition Network Graph PD Maximum Persistence - P(D)',
    'Ordinal graph $E\'(D)$': 'Ordinal Partition Network Graph PD Normalised Persistent Entropy - E\'(D)',
    'Rosenstein': 'Rosenstein Scaling',
    'Eckmann': 'Eckmann Scaling',
    'Kantz': 'Kantz Scaling',
    'cophenetic': 'PLMT Cophenetic Divergence',
    'cophenetic_reverse': 'PLMT Cophenetic (Reverse) Divergence',
    'cophenetic_edge': 'PLMT Cophenetic Divergence (Edge-Normalised)',
    'cophenetic_reverse_edge': 'PLMT Cophenetic (Reverse) Divergence (Edge-Normalised)',
    '$\\Delta$VGA ($L_1$)': 'HVG Top-Bottom Degree Distribution L1 Distance',
    '$\\Delta$VGA ($W_1$)': 'HVG Top-Bottom Degree Distribution Wasserstein Distance',    
    'Betti Vector L1 Norm': 'Persistence Diagram Betti Vector L1 Norm',
}

measure_short_name_map = {
    'cophenetic divergence': 'DMT Cophenetic',
    'cophenetic_reverse divergence': 'DMT Cophenetic (Reverse)',
    'interleaving_0.5_None divergence': 'PLMT Interleaving (0.5)',
    'interleaving_0.4_None divergence': 'PLMT Interleaving (0.4)',
    'interleaving_0.3_None divergence': 'PLMT Interleaving (0.3)',
    'interleaving_0.2_None divergence': 'PLMT Interleaving (0.2)',
    'interleaving_0.1_None divergence': 'PLMT Interleaving (0.1)',
    'interleaving_edge_0.5_None divergence': 'PLMT Interleaving (0.5) / Edges',
    'interleaving_edge_0.4_None divergence': 'PLMT Interleaving (0.4) / Edges',
    'interleaving_edge_0.3_None divergence': 'PLMT Interleaving (0.3) / Edges',
    'interleaving_edge_0.2_None divergence': 'PLMT Interleaving (0.2) / Edges',
    'interleaving_edge_0.1_None divergence': 'PLMT Interleaving (0.1) / Edges',
    'bottleneck divergence': 'PD Bottleneck',
    'wasserstein divergence': 'PD Wasserstein',
    '$k$-NN graph $M(D)$': 'kNN Homology Class Ratio',
    '$k$-NN graph $P(D)$': 'kNN Maximum Persistence',
    '$k$-NN graph $E\'(D)$': 'kNN Persistent Entropy',
    'Ordinal graph $M(D)$': 'OPN Homology Class Ratio',
    'Ordinal graph $P(D)$': 'OPN Maximum Persistence',
    'Ordinal graph $E\'(D)$': 'OPN Persistent Entropy',
    'Rosenstein': 'Rosenstein',
    'Eckmann': 'Eckmann',
    'Kantz': 'Kantz',
    'cophenetic': 'PLMT Cophenetic',
    'cophenetic_reverse': 'PLMT Cophenetic (Reverse)',
    'cophenetic_edge': 'PLMT Cophenetic / Edges',
    'cophenetic_reverse_edge': 'PLMT Cophenetic (Reverse) / Edges',
    '$\\Delta$VGA ($L_1$)': 'HVG L1',
    '$\\Delta$VGA ($W_1$)': 'HVG W1',
    'Betti Vector L1 Norm': 'Betti L1',
}

measure_type_map = {
    'cophenetic divergence': 'DMT Cophenetic',
    'cophenetic_reverse divergence': 'DMT Cophenetic',
    'interleaving_0.5_None divergence': 'PLMT Interleaving',
    'interleaving_0.4_None divergence': 'PLMT Interleaving',
    'interleaving_0.3_None divergence': 'PLMT Interleaving',
    'interleaving_0.2_None divergence': 'PLMT Interleaving',
    'interleaving_0.1_None divergence': 'PLMT Interleaving',
    'interleaving_edge_0.5_None divergence': 'PLMT Interleaving (Normalised)',
    'interleaving_edge_0.4_None divergence': 'PLMT Interleaving (Normalised)',
    'interleaving_edge_0.3_None divergence': 'PLMT Interleaving (Normalised)',
    'interleaving_edge_0.2_None divergence': 'PLMT Interleaving (Normalised)',
    'interleaving_edge_0.1_None divergence': 'PLMT Interleaving (Normalised)',
    'bottleneck divergence': 'TDA Divergence',
    'wasserstein divergence': 'TDA Divergence',
    '$k$-NN graph $M(D)$': 'TDA Embedding',
    '$k$-NN graph $P(D)$': 'TDA Embedding',
    '$k$-NN graph $E\'(D)$': 'TDA Embedding',
    'Ordinal graph $M(D)$': 'TDA Embedding',
    'Ordinal graph $P(D)$': 'TDA Embedding',
    'Ordinal graph $E\'(D)$': 'TDA Embedding',
    'Rosenstein': 'Classical Embedding',
    'Eckmann': 'Classical Embedding',
    'Kantz': 'Classical Embedding',
    'cophenetic': 'PLMT Cophenetic',
    'cophenetic_reverse': 'PLMT Cophenetic',
    'cophenetic_edge': 'PLMT Cophenetic',
    'cophenetic_reverse_edge': 'PLMT Cophenetic',
    '$\\Delta$VGA ($L_1$)': 'HVG Graph Distance',
    '$\\Delta$VGA ($W_1$)': 'HVG Graph Distance',
    'Betti Vector L1 Norm': 'TDA Direct',
}

In [4]:
# add measure type column
get_type = lambda row: measure_type_map[row['Measure']]
merged_data['Measure Type'] = merged_data.apply(get_type, axis=1)

In [5]:
# standardise measure names with more useful info
for k, v in measure_short_name_map.items():
    merged_data["Measure"] = merged_data["Measure"].replace(k, v)


In [6]:
merged_data.head()

,Measure,System,Length,Random Forest F1 Score,Random Forest MSE (All),Random Forest MSE (Chaos),kNN MSE (All),kNN MSE (Chaos),Spearman Rho (All),Spearman p (All),Spearman Rho (Chaos),Spearman p (Chaos),Measure Type
0,DMT Cophenetic,Henon,500,0.770146,-0.088087,-0.065053,-0.085484,-0.059962,0.736482,1.396494e-86,0.746799,3.699274e-43,DMT Cophenetic
1,DMT Cophenetic (Reverse),Henon,500,0.784740,-0.118038,-0.093484,-0.093558,-0.067900,0.798787,5.628399e-112,0.831906,1.592406e-61,DMT Cophenetic
2,PLMT Interleaving (0.5),Henon,500,0.779450,-0.094765,-0.092251,-0.085155,-0.076143,0.795286,2.552484e-110,0.606278,5.620223e-25,PLMT Interleaving
4,PLMT Interleaving (0.5) / Edges,Henon,500,0.782874,-0.102633,-0.085643,-0.090005,-0.078395,0.790853,2.875452e-108,0.699864,6.632495e-36,PLMT Interleaving (Normalised)
5,PLMT Interleaving (0.4),Henon,500,0.790967,-0.119352,-0.084975,-0.086889,-0.076964,0.769395,5.357184e-99,0.585371,5.242113e-23,PLMT Interleaving


## Regression Performance: Predicting Largest Lyapunov Exponent $\lambda_{\max}$ in the Chaotic Regime

This table reports results of $k$-nearest neighbour regression using $k=10$ neighbours and uniform distance weighting of each neighbour. Values reported in the four system columns are means of (negative) mean squared error (MSE) values over 10 train-test splits of chaotic trajectories of length 500, with the corresponding largest Lyapunov exponent (LLE) as ground truth, from the given systems. Control parameter values generating each system trajectory are sampled uniformly from pre-chosen ranges to ensure inclusion of samples close to bifurcation points. Ground truth LLEs were calculated using Benettin's Jacobian-matrix-based algorithm, on which basis trajectories were then filtered, with those satisfying $\lambda_{\max}>0$ retained. This provided a wide range of representative positive $\lambda_{\max}$ values for each of the systems.

The _Overall Rank_ column gives the overall ranked performance of each measure across all the systems considered.

The best performing feature for prediction of $\lambda_{\max}$ for short-length time series is the _discrete merge tree cophenetic divergence_. This outperforms the well-established and widely used Kantz algorithm, which ranks second, and it achieves lower variance in its respective performance.

In [8]:
df = merged_data
filtered_df = df[df['Length'] == 500]
knn_mse_pivot_table = pd.pivot_table(filtered_df, values='kNN MSE (Chaos)', index=['Measure Type', 'Measure'], columns=['System'], aggfunc=lambda x: x)
rank_columns = []
for col in knn_mse_pivot_table.columns:
    rank_col_name = f"{col} Rank"
    knn_mse_pivot_table[rank_col_name] = knn_mse_pivot_table[col].rank(method='min', ascending=False).astype(int)
    rank_columns.append(rank_col_name)
knn_mse_pivot_table['Mean Rank'] = knn_mse_pivot_table[rank_columns].mean(axis=1)
knn_mse_pivot_table['STD Rank'] = knn_mse_pivot_table[rank_columns].std(axis=1)
knn_mse_pivot_table['Overall Rank'] = knn_mse_pivot_table['Mean Rank'].rank(method='min', ascending=True).astype(int)
pd.options.display.float_format = '{:,.3f}'.format
display(knn_mse_pivot_table)

System                                                            Henon  \
Measure Type                   Measure                                    
Classical Embedding            Eckmann                           -0.165   
                               Kantz                             -0.023   
                               Rosenstein                        -0.097   
DMT Cophenetic                 DMT Cophenetic                    -0.060   
                               DMT Cophenetic (Reverse)          -0.068   
HVG Graph Distance             HVG L1                            -0.115   
                               HVG W1                            -0.088   
PLMT Cophenetic                PLMT Cophenetic                   -0.047   
                               PLMT Cophenetic (Reverse)         -0.043   
                               PLMT Cophenetic (Reverse) / Edges -0.073   
                               PLMT Cophenetic / Edges           -0.080   
PLMT Interleaving              PLMT Interleaving (0.1)           -0.066   
                               PLMT Interleaving (0.2)           -0.112   
                               PLMT Interleaving (0.3)           -0.064   
                               PLMT Interleaving (0.4)           -0.077   
                               PLMT Interleaving (0.5)           -0.076   
PLMT Interleaving (Normalised) PLMT Interleaving (0.1) / Edges   -0.063   
                               PLMT Interleaving (0.2) / Edges   -0.108   
                               PLMT Interleaving (0.3) / Edges   -0.083   
                               PLMT Interleaving (0.4) / Edges   -0.078   
                               PLMT Interleaving (0.5) / Edges   -0.078   
TDA Direct                     Betti L1                          -0.072   
TDA Divergence                 PD Bottleneck                     -0.110   
                               PD Wasserstein                    -0.122   
TDA Embedding                  OPN Homology Class Ratio          -0.031   
                               OPN Maximum Persistence           -0.048   
                               OPN Persistent Entropy            -0.077   
                               kNN Homology Class Ratio          -0.037   
                               kNN Maximum Persistence           -0.088   
                               kNN Persistent Entropy            -0.085   

System                                                            Ikeda  \
Measure Type                   Measure                                    
Classical Embedding            Eckmann                           -0.072   
                               Kantz                             -0.095   
                               Rosenstein                        -0.120   
DMT Cophenetic                 DMT Cophenetic                    -0.069   
                               DMT Cophenetic (Reverse)          -0.076   
HVG Graph Distance             HVG L1                            -0.108   
                               HVG W1                            -0.123   
PLMT Cophenetic                PLMT Cophenetic                   -0.135   
                               PLMT Cophenetic (Reverse)         -0.129   
                               PLMT Cophenetic (Reverse) / Edges -0.108   
                               PLMT Cophenetic / Edges           -0.110   
PLMT Interleaving              PLMT Interleaving (0.1)           -0.083   
                               PLMT Interleaving (0.2)           -0.075   
                               PLMT Interleaving (0.3)           -0.068   
                               PLMT Interleaving (0.4)           -0.074   
                               PLMT Interleaving (0.5)           -0.077   
PLMT Interleaving (Normalised) PLMT Interleaving (0.1) / Edges   -0.078   
                               PLMT Interleaving (0.2) / Edges   -0.071   
                               PLMT Interleaving (0.3) / Edges   -0.061   
                        

In [215]:
print(knn_mse_pivot_table.to_latex(float_format="%.4f"))

\begin{tabular}{llrrrrrrrrrrr}
\toprule
              & System &   Henon &   Ikeda &  Logistic &  Tinkerbell &  Henon Rank &  Ikeda Rank &  Logistic Rank &  Tinkerbell Rank &  Mean Rank &  STD Rank &  Place \\
Measure Type & Measure &         &         &           &             &             &             &                &                  &            &           &        \\
\midrule
Classical Embedding & Eckmann & -0.1654 & -0.0721 &   -0.2270 &     -0.0331 &          29 &           6 &             29 &               22 &    21.5000 &   10.8474 &     25 \\
              & Kantz & -0.0231 & -0.0950 &   -0.0604 &     -0.0159 &           1 &          17 &              5 &                2 &     6.2500 &    7.3655 &      2 \\
              & Rosenstein & -0.0967 & -0.1198 &   -0.1589 &     -0.0157 &          23 &          26 &             27 &                1 &    19.2500 &   12.2848 &     22 \\
DMT Cophenetic & DMT Cophenetic & -0.0600 & -0.0691 &   -0.0636 &     -0.0197 &           7

/var/folders/7h/nhqr9y0j09b78bvjy9hlwfhw0000gn/T/ipykernel_73859/2991495372.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(knn_mse_pivot_table.to_latex(float_format="%.4f"))


## Statistical Dependence Performance

### Spearman's $\rho$ in the Chaotic Regime

Correlation results for the studied measures appear in the table. Values reported in the system columns are Spearman's $\rho$ coefficient, measuring the rank correlation, or strength of monotonic (but possibly non-linear) dependence, between the studied measures and the actual system $\lambda_{\max}$ value. Corresponding $p$-values are generally of the order of $10^{-20}$ or less whenever $|\rho|>0.5$. Trajectory length is 500.

The most highly correlated measures with $\lambda_{\max}$, for the short-length time series considered, are the _discrete merge tree cophenetic divergence_ and its reversed variant (in which the order of leaves in the superlevel tree is reversed for the divergence calculation). These are both discrete merge tree topological divergences. The next most highly correlated group of measures are those in the piecewise linear merge tree interleaving divergences. Despite showing less monotonic dependence on $\lambda_{\max}$ than the discrete merge tree measures do, these are more strongly correlated with $\lambda_{\max}$ than all other existing measures.

### Correlation across all dynamic regimes

It should be noted that the classical embedding-based estimators of Eckmann, Kantz, and Rosenstein are less highly correlated with $\lambda_{\max}$ in the chaotic regime, for the time series considered, than topological divergences are. However, the former estimators are more highly correlated with $\lambda_{\max}$ when non-chaotic sequences are included. Topological divergences are therefore not a general purpose measure for $\lambda_{\max}$ when the true value of the latter may be negative. A split pipeline of chaos detection (classification) followed by Lyapunov exponent estimation using either topological divergences (when chaos has been identified) or other estimators (when chaos is not present) is likely to be more effective than using one or other type of measure exclusively.


In [9]:
df = merged_data
filtered_df = df[df['Length'] == 500]
spearman_rho_pivot_table = pd.pivot_table(filtered_df, values='Spearman Rho (Chaos)', index=['Measure Type', 'Measure'], columns=['System'], aggfunc=lambda x: x)
rank_columns = []
for col in spearman_rho_pivot_table.columns:
    rank_col_name = f"{col} Rank"
    spearman_rho_pivot_table[rank_col_name] = spearman_rho_pivot_table[col].abs().rank(method='min', ascending=False).astype(int)
    rank_columns.append(rank_col_name)
spearman_rho_pivot_table['Mean Rank'] = spearman_rho_pivot_table[rank_columns].mean(axis=1)
spearman_rho_pivot_table['STD Rank'] = spearman_rho_pivot_table[rank_columns].std(axis=1)
spearman_rho_pivot_table['Place'] = spearman_rho_pivot_table['Mean Rank'].rank(method='min', ascending=True).astype(int)
display(spearman_rho_pivot_table)

System                                                            Henon  \
Measure Type                   Measure                                    
Classical Embedding            Eckmann                           -0.117   
                               Kantz                              0.753   
                               Rosenstein                         0.235   
DMT Cophenetic                 DMT Cophenetic                     0.747   
                               DMT Cophenetic (Reverse)           0.832   
HVG Graph Distance             HVG L1                             0.439   
                               HVG W1                             0.366   
PLMT Cophenetic                PLMT Cophenetic                    0.733   
                               PLMT Cophenetic (Reverse)          0.745   
                               PLMT Cophenetic (Reverse) / Edges  0.816   
                               PLMT Cophenetic / Edges            0.807   
PLMT Interleaving              PLMT Interleaving (0.1)            0.703   
                               PLMT Interleaving (0.2)            0.431   
                               PLMT Interleaving (0.3)            0.618   
                               PLMT Interleaving (0.4)            0.585   
                               PLMT Interleaving (0.5)            0.606   
PLMT Interleaving (Normalised) PLMT Interleaving (0.1) / Edges    0.699   
                               PLMT Interleaving (0.2) / Edges    0.643   
                               PLMT Interleaving (0.3) / Edges    0.695   
                               PLMT Interleaving (0.4) / Edges    0.676   
                               PLMT Interleaving (0.5) / Edges    0.700   
TDA Direct                     Betti L1                          -0.761   
TDA Divergence                 PD Bottleneck                     -0.090   
                               PD Wasserstein                     0.243   
TDA Embedding                  OPN Homology Class Ratio           0.787   
                               OPN Maximum Persistence            0.540   
                               OPN Persistent Entropy             0.506   
                               kNN Homology Class Ratio           0.659   
                               kNN Maximum Persistence            0.489   
                               kNN Persistent Entropy            -0.140   

System                                                            Ikeda  \
Measure Type                   Measure                                    
Classical Embedding            Eckmann                           -0.825   
                               Kantz                              0.329   
                               Rosenstein                        -0.560   
DMT Cophenetic                 DMT Cophenetic                     0.876   
                               DMT Cophenetic (Reverse)           0.887   
HVG Graph Distance             HVG L1                             0.178   
                               HVG W1                             0.197   
PLMT Cophenetic                PLMT Cophenetic                    0.043   
                               PLMT Cophenetic (Reverse)          0.062   
                               PLMT Cophenetic (Reverse) / Edges  0.437   
                               PLMT Cophenetic / Edges            0.416   
PLMT Interleaving              PLMT Interleaving (0.1)            0.781   
                               PLMT Interleaving (0.2)            0.825   
                               PLMT Interleaving (0.3)            0.846   
                               PLMT Interleaving (0.4)            0.850   
                               PLMT Interleaving (0.5)            0.824   
PLMT Interleaving (Normalised) PLMT Interleaving (0.1) / Edges    0.832   
                               PLMT Interleaving (0.2) / Edges    0.856   
                               PLMT Interleaving (0.3) / Edges    0.878   
                        